<a href="https://colab.research.google.com/github/z-gard/analysis/blob/main/notebooks/hospital_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 医療機関
- [診療・検査医療機関の一覧リスト](https://catalog.data.metro.tokyo.lg.jp/dataset/t000010d0000000095/resource/49aa4ba8-32e6-4e35-ad85-0cdb4db01950)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse, quote
import io
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [4]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [5]:
def get_data_csv_url(url, encoding='utf-8', excel=False):
    res = requests.get(url)
    if res.status_code == 200:
        if not excel:
            return pd.read_csv(io.BytesIO(res.content), encoding=encoding)
        else:
            return pd.read_excel(io.BytesIO(res.content))
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

In [43]:
hospital_url = 'https://www.opendata.metro.tokyo.lg.jp/fukushihoken/130001_shinryoukensa.csv'

df_hospital = get_data_csv_url(hospital_url)
print(df_hospital.shape)

(4759, 40)


In [44]:
df_hospital['施設分類'] = '医療機関'
df_hospital = df_hospital[df_hospital['区市町村'].str.endswith('区')].rename(columns={'医療機関名':'名称', '区市町村':'市区町村名', '正規化住所':'所在地'})\
    [['検索キー', '名称', '市区町村名', '所在地', '施設分類', '小児', '妊婦']].reset_index(drop=True)
print(df_hospital.shape)

(3596, 7)


In [45]:
df_hospital.head()

,検索キー,名称,市区町村名,所在地,施設分類,小児,妊婦
0,メディカクリニック神田小川町3-1-10 メディカビル4階,メディカクリニック,千代田区,東京都千代田区神田小川町3-1-10,医療機関,1.0,1.0
1,九段坂内科消化器内科クリニック神田神保町3丁目2-3Daiwa神保町3丁目ビル,九段坂内科消化器内科クリニック,千代田区,東京都千代田区神田神保町3丁目2-3Daiwa神保町3丁目ビル,医療機関,NaN,NaN
2,医療法人社団平晴会晴海3丁目クリニック晴海3-10-1 Daiwa晴海ビル1F,医療法人社団平晴会晴海3丁目クリニック,中央区,東京都中央区晴海3-10-1,医療機関,NaN,NaN
3,文京春日こどもクリニック小石川1－5－1 パークコート文京小石川ザタワー304,文京春日こどもクリニック,文京区,東京都文京区小石川1－5－1,医療機関,2.0,NaN
4,蔵前まえだクリニック寿1-2-3 ATビル３階,蔵前まえだクリニック,台東区,東京都台東区寿1-2-3,医療機関,NaN,NaN


### 場所を探す

In [18]:
YOUR_API_KEY = 'xxxxxx'

In [17]:
def get_place_info(place_name, unique_no, is_rating=True):
    place = quote(place_name)
    rating = '%2Crating' if is_rating else ''
    
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}&inputtype=textquery&fields=name{rating}%2Cgeometry&key={YOUR_API_KEY}"
    response = requests.get(url)
    if response.status_code == requests.codes.ok:
        try:
            _df = pd.DataFrame(response.json()['candidates'])
            _df['no'] = unique_no
            _df['place_name'] = place_name
            _df['lat'] = _df['geometry'].apply(lambda x: x['location']['lat'])
            _df['lng'] = _df['geometry'].apply(lambda x: x['location']['lng'])
            if not is_rating:
                _df['rating'] = np.NaN
            return _df[['no', 'place_name', 'name', 'lat', 'lng', 'rating']]
        except Exception as e:
            print(unique_no, place_name, e)
            pass
    else:
        print(unique_no, place_name, response.status_code)
    
    return pd.DataFrame([{'no': unique_no, 'place_name': place_name}])

In [22]:
df_result = pd.DataFrame()
for index, row in df_hospital.iterrows():
    _df = get_place_info(row['検索キー'], index)
    df_result = pd.concat([df_result, _df])
df_result = df_result.reset_index(drop=True)
print(df_result.shape)

106 近藤しんたろうクリニック松涛1-28-7-201 "['rating'] not in index"
126 医療法人社団 アースクリニック池袋西池袋3-25-10リバーストンビル3階 'geometry'
206 東京メディカルテラス北青山2－13－4　青山MYビル4階 'geometry'
259 社会医療法人社団大成会　長汐病院池袋1-5-8 "['rating'] not in index"
262 kクリニック要町1－30－1 'geometry'
467 社会福祉法人聖母会 聖母病院中落合二丁目5番1号 "['rating'] not in index"
514 赤池耳鼻咽喉科千石４－４０－２ "['rating'] not in index"
730 愛和病院東砂4-20-2 "['rating'] not in index"
734 三ツ木診療所西品川2-13-20 "['rating'] not in index"
738 鈴木内科医院東大井6-7-3 "['rating'] not in index"
756 医療法人 社団 笑鯨会 なかのぶクリニック東中延2-10-11 智勇第二ビル3階 'geometry'
884 大田病院附属うのき診療所鵜の木3-13-10 "['rating'] not in index"
938 水野内科クリニック池上6-11-16ワキタビル2F 'geometry'
947 医療法人社団静恒会 本多病院東矢口一丁目17-15 "['rating'] not in index"
1078 橋本医院野方3-15-3 "['rating'] not in index"
1080 山田クリニック南台5-32-2 "['rating'] not in index"
1269 たかねクリニック中里３－１７－１ "['rating'] not in index"
1273 医療法人社団慈広会 王子神谷齋藤クリニック豊島8-24-4 "['rating'] not in index"
1326 医療法人社団健康会 高橋医院町屋三丁目2番10号ISEビル1階2階3階地下1階 'geometry'
1439 医療法人社団 明芳会 イムス記念病院常盤台4-25-5 "['rating'] not in index"
1517 山東

### データが取得できなかった病院

In [25]:
df_na_data = df_result[df_result['name'].isna()][['no', 'place_name']]
print(len(df_na_data))

50


In [26]:
# ratingなしで検索
df_result_2 = pd.DataFrame()
for index, row in df_na_data.iterrows():
    _df = get_place_info(row['place_name'], row['no'], is_rating=False)
    df_result_2 = pd.concat([df_result_2, _df])
df_result_2 = df_result_2.reset_index(drop=True)
print(df_result_2.shape)

126 医療法人社団 アースクリニック池袋西池袋3-25-10リバーストンビル3階 'geometry'
206 東京メディカルテラス北青山2－13－4　青山MYビル4階 'geometry'
262 kクリニック要町1－30－1 'geometry'
756 医療法人 社団 笑鯨会 なかのぶクリニック東中延2-10-11 智勇第二ビル3階 'geometry'
938 水野内科クリニック池上6-11-16ワキタビル2F 'geometry'
1326 医療法人社団健康会 高橋医院町屋三丁目2番10号ISEビル1階2階3階地下1階 'geometry'
1562 医療法人社団医眞会　竹ノ塚診療所竹の塚1-39-14-101 'geometry'
2043 あおきクリニック浜松町1-9-12中野ビル5階 'geometry'
2126 医療法人社団ホームアレー　ホームアレークリニック不動前西五反田3-15-8　ビズフィールド目黒9F 'geometry'
2164 医療法人社団健栄会　渋谷駅東口ひろばクリニック渋谷1-24-15FPG links SHIBUYA9階 'geometry'
2196 森　医院中台2-17-1-213 'geometry'
2385 医療法人社団 相心会 ワイズクリニック南麻布4-13-7 南麻布アコルトビル5階 'geometry'
2515 医療法人社団パリアン クリニック川越立川2-1-9 KHハウス3F 'geometry'
2994 医療法人社団ききょう会東十条クリニック東十条2-14-11マンシオンフルーランス102 'geometry'
3000 医療法人社団 天使の梯子 十条堀江クリニック東京都北区十条仲原2-10-20 'geometry'
3002 医療法人社団あおば　つばめ医科歯科クリニック東田端１－２－８　早川住建田端マンション１０１号室 'geometry'
3017 医療法人社団健咲樹会東尾久8-14-1-2F 'geometry'
3019 医療法人社団ことり会 ことり訪問診療所東日暮里５－５２－６折原ビル４階 'geometry'
3051 上板橋松山内科クリニック常盤台4-30-9宝田ビル1F 'geometry'
3074 工藤クリニック高島平二丁目32街区2号棟102号 'geometry'
3075

### 重複している検索結果を削除して結合

In [32]:
df_result_all = df_result.drop_duplicates(subset=['no', 'place_name'])
print(len(df_result_all))

df_result_all = pd.concat([
    df_result_all[df_result_all['name'].notna()],
    df_result_2[df_result_2['name'].notna()]
])
print(df_result_all.shape)

3596
(3571, 6)


In [37]:
assert df_result_all['no'].nunique() == len(df_result_all)

In [56]:
df_hospital_master = pd.concat([
    df_hospital[['名称', '市区町村名', '所在地', '施設分類', '小児', '妊婦']],
    df_result_all.set_index('no')[['lat', 'lng', 'rating']]
], axis=1).sort_index()

In [58]:
df_hospital_master = df_hospital_master[
    (df_hospital_master['lat'].notna())&(df_hospital_master['lng'].notna())
].rename(columns={'lat':'緯度', 'lng':'経度'}).reset_index(drop=True)
print(df_hospital_master.shape)

(3571, 9)


In [59]:
df_hospital_master

,名称,市区町村名,所在地,施設分類,小児,妊婦,緯度,経度,rating
0,メディカクリニック,千代田区,東京都千代田区神田小川町3-1-10,医療機関,1.0,1.0,35.694355,139.762639,3.6
1,九段坂内科消化器内科クリニック,千代田区,東京都千代田区神田神保町3丁目2-3Daiwa神保町3丁目ビル,医療機関,NaN,NaN,35.696069,139.754469,4.2
2,医療法人社団平晴会晴海3丁目クリニック,中央区,東京都中央区晴海3-10-1,医療機関,NaN,NaN,35.654056,139.779257,4.7
3,文京春日こどもクリニック,文京区,東京都文京区小石川1－5－1,医療機関,2.0,NaN,35.710461,139.752461,4.2
4,蔵前まえだクリニック,台東区,東京都台東区寿1-2-3,医療機関,NaN,NaN,35.706362,139.789713,4.6
...,...,...,...,...,...,...,...,...,...
3566,にれの木クリニック,杉並区,東京都杉並区天沼2-3-18カシタスデ荻窪107,医療機関,1.0,1.0,35.705432,139.624899,2.4
3567,mocoこどもクリニック,杉並区,東京都杉並区久我山４－５０－２,医療機関,2.0,NaN,35.689386,139.599999,3.8
3568,いたるクリニック,杉並区,東京都杉並区阿佐谷南2-14-8,医療機関,NaN,NaN,35.704418,139.637852,0.0
3569,独立行政法人国立病院機構東京医療センター,目黒区,東京都目黒区東が丘２－５－１,医療機関,NaN,NaN,35.626213,139.666690,3.2


In [60]:
df_hospital_master.to_csv(os.path.join(DIR_NAME, 'hospital_master.csv'), index=False)